In [38]:
# -*- coding: utf-8 -*-
import pandas as pd
import warnings

# Bỏ qua (ignore) các cảnh báo loại PerformanceWarning từ Pandas
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import calendar
import numpy as np
import time  # Thêm để đo thời gian (tùy chọn)

# Biến toàn cục để lưu dữ liệu đã xử lý
df_all = pd.DataFrame()
df_sanluong = pd.DataFrame()
df_sanluong_indexed = pd.DataFrame()
nmtd_cache = {}
madiemdo_cache = {}
_observed_callbacks = {}
ui_components = {}
parquet_folder = r"C:\Khue\TDN\data\processed"


def read_data(parquet_folder):
    """Đọc dữ liệu từ các file Parquet"""
    global df_all

    print("🔄 Bắt đầu đọc dữ liệu Parquet...")
    start_read_time = time.time()

    try:
        all_files_in_folder = os.listdir(parquet_folder)
        parquet_files = [
            os.path.join(parquet_folder, f)
            for f in all_files_in_folder
            if f.startswith("sanluong_") and f.endswith(".parquet")
        ]

        if not parquet_files:
            print(f"⚠️ Không tìm thấy file Parquet trong: {parquet_folder}")
            df_all = pd.DataFrame()
        else:
            df_list = []
            print(f"🔍 Tìm thấy {len(parquet_files)} file Parquet. Bắt đầu đọc...")
            for i, f in enumerate(parquet_files):
                try:
                    df_temp = pd.read_parquet(f, engine="pyarrow")
                    # Chỉ đọc các cột thực sự cần thiết ngay từ đầu để tiết kiệm bộ nhớ
                    required_cols_read = ["CTDL", "NMTD", "MADIEMDO", "ENDTIME", "CS"]
                    if all(col in df_temp.columns for col in required_cols_read):
                        df_list.append(
                            df_temp[required_cols_read]
                        )  # Chỉ lấy cột cần thiết
                    else:
                        print(f"   ⚠️ File {os.path.basename(f)} thiếu cột, bỏ qua.")
                except Exception as e:
                    print(f"❌ Lỗi đọc file {os.path.basename(f)}: {e}")

            if df_list:
                print("   Ghép các DataFrame...")
                df_all = pd.concat(df_list, ignore_index=True)
                print(f"✅ Đọc và ghép {len(df_list)} file thành công.")
                print(f"👉 Tổng số dòng: {df_all.shape[0]:,}")
                print(
                    f"⏱️ Thời gian đọc và ghép: {time.time() - start_read_time:.2f} giây"
                )
            else:
                print("❌ Không đọc được file nào thành công.")
                df_all = pd.DataFrame()

    except FileNotFoundError:
        print(f"❌ Lỗi: Không tìm thấy thư mục: {parquet_folder}")
        df_all = pd.DataFrame()
    except Exception as e:
        print(f"❌ Lỗi không xác định khi đọc file/thư mục: {e}")
        df_all = pd.DataFrame()

    return df_all


def preprocess_data():
    """Tiền xử lý và tối ưu hóa dữ liệu"""
    global df_all, df_sanluong, df_sanluong_indexed

    df_sanluong = pd.DataFrame()
    df_sanluong_indexed = pd.DataFrame()

    if not df_all.empty:
        print("\n🔄 Bắt đầu tiền xử lý và tối ưu hóa...")
        start_process_time = time.time()
        try:
            # 1. Đổi tên cột và đảm bảo kiểu dữ liệu datetime
            df_sanluong = df_all.rename(
                columns={"ENDTIME": "TIME"}, errors="raise"
            )  # Đảm bảo cột tồn tại
            df_sanluong["TIME"] = pd.to_datetime(df_sanluong["TIME"])
            
            # Thêm bước lọc các dòng dữ liệu có thời gian không theo đúng chuẩn 30 phút
            initial_rows_before_time_filter = len(df_sanluong)

            # Lọc các dòng dữ liệu có phút là 0 hoặc 30 và giây là 0
            valid_time_mask = (
                (df_sanluong["TIME"].dt.minute == 0)
                | (df_sanluong["TIME"].dt.minute == 30)
            ) & (df_sanluong["TIME"].dt.second == 0)

            df_sanluong = df_sanluong[valid_time_mask]

            removed_time_rows = initial_rows_before_time_filter - len(df_sanluong)
            if removed_time_rows > 0:
                print(
                    f"   ⚠️ Đã loại bỏ {removed_time_rows:,} dòng có thời gian không theo chuẩn 30 phút."
                )
                
            # 2. Đảm bảo kiểu dữ liệu số cho CS và xử lý NaN
            df_sanluong["CS"] = pd.to_numeric(df_sanluong["CS"], errors="coerce")
            initial_rows = len(df_sanluong)
            df_sanluong.dropna(subset=["CS"], inplace=True)
            removed_rows = initial_rows - len(df_sanluong)
            if removed_rows > 0:
                print(
                    f"   ⚠️ Đã loại bỏ {removed_rows:,} dòng có giá trị CS không hợp lệ."
                )

            # 3. Tạo các cột thời gian cần thiết
            df_sanluong["YEAR"] = df_sanluong["TIME"].dt.year.astype("int16")
            df_sanluong["MONTH_NUM"] = df_sanluong["TIME"].dt.month.astype("int8")
            df_sanluong["DAY_NUM"] = df_sanluong["TIME"].dt.day.astype("int8")
            df_sanluong["TIME_SLOT"] = df_sanluong["TIME"].dt.strftime("%H:%M")

            # 4. Tối ưu kiểu dữ liệu Categorical cho các cột chuỗi lặp lại
            df_sanluong["CTDL"] = df_sanluong["CTDL"].astype("category")
            df_sanluong["NMTD"] = df_sanluong["NMTD"].astype("category")
            df_sanluong["MADIEMDO"] = df_sanluong["MADIEMDO"].astype("category")
            df_sanluong["TIME_SLOT"] = df_sanluong["TIME_SLOT"].astype("category")

            print("   ✅ DataFrame gốc đã được xử lý và tối ưu hóa kiểu dữ liệu.")

            # 5. Tạo DataFrame được Index cho tính toán thống kê nhanh
            index_cols = ["CTDL", "NMTD", "MADIEMDO", "YEAR", "MONTH_NUM", "DAY_NUM"]
            # Chỉ cần TIME_SLOT và CS cho df_indexed
            df_sanluong_indexed = df_sanluong[index_cols + ["TIME_SLOT", "CS"]].copy()
            # Giữ kiểu category cho index để tiết kiệm bộ nhớ index
            df_sanluong_indexed.set_index(index_cols, inplace=True)
            df_sanluong_indexed.sort_index(
                inplace=True
            )  # Rất quan trọng cho hiệu năng loc
            print("   ✅ DataFrame index cho thống kê đã được tạo và sắp xếp.")

            print(
                f"⏱️ Thời gian tiền xử lý: {time.time() - start_process_time:.2f} giây"
            )

        except Exception as e:
            print(f"❌ Lỗi trong quá trình tiền xử lý: {e}")
            # Đảm bảo cả hai df đều rỗng nếu có lỗi
            df_sanluong = pd.DataFrame()
            df_sanluong_indexed = pd.DataFrame()
    else:
        print("ℹ️ Không có dữ liệu đầu vào để xử lý.")

    return df_sanluong, df_sanluong_indexed


def create_ui():
    """Tạo giao diện tương tác"""
    global df_sanluong, df_sanluong_indexed, ui_components, _observed_callbacks
    global nmtd_cache, madiemdo_cache

    if not df_sanluong.empty and not df_sanluong_indexed.empty:
        print("\n🔄 Tạo giao diện tương tác...")
        # ========== DANH SÁCH CHO DROPDOWN ==========
        try:
            ctdl_list = sorted(df_sanluong["CTDL"].cat.categories)
            year_list = sorted(df_sanluong["YEAR"].unique())
            nmtd_list_all = sorted(df_sanluong["NMTD"].cat.categories)
        except Exception as e:
            print(f"❌ Lỗi lấy danh sách dropdown: {e}")
            ctdl_list, year_list, nmtd_list_all = [], [], []

        # ========== TẠO DROPDOWNS ==========
        # Tab 1
        ctdl_dropdown_month_plot = make_dropdown(ctdl_list, "CTDL:", "300px")
        nmtd_dropdown_month_plot = make_dropdown([], "Nhà máy:", "300px")
        year_dropdown_month_plot = make_dropdown(year_list, "Năm:", "120px")
        month_dropdown_month_plot = make_dropdown(list(range(1, 13)), "Tháng:", "120px")
        placeholder_month_plot = widgets.Label(
            value="", layout=widgets.Layout(width="120px", margin="0px 20px 0px 0px")
        )
        # Tab 2
        ctdl_dropdown_day_plot = make_dropdown(ctdl_list, "CTDL:", "300px")
        nmtd_dropdown_day_plot = make_dropdown([], "Nhà máy:", "300px")
        year_dropdown_day_plot = make_dropdown(year_list, "Năm:", "120px")
        month_dropdown_day_plot = make_dropdown(list(range(1, 13)), "Tháng:", "120px")
        day_dropdown_day_plot = make_dropdown([], "Ngày:", "120px")
        # Tab 3
        ctdl_dropdown_stat = make_dropdown(ctdl_list, "CTDL:", "300px")
        nmtd_dropdown_stat = make_dropdown([], "Nhà máy:", "300px")
        madiemdo_dropdown_stat = make_dropdown([], "Mã điểm đo:", "300px")
        year_dropdown_stat = make_dropdown(year_list, "Năm:", "120px")
        month_dropdown_stat = make_dropdown(list(range(1, 13)), "Tháng:", "120px")

        # ========== NHÓM WIDGET ==========
        month_plot_widgets = [
            ctdl_dropdown_month_plot,
            nmtd_dropdown_month_plot,
            year_dropdown_month_plot,
            month_dropdown_month_plot,
        ]
        day_plot_widgets = [
            ctdl_dropdown_day_plot,
            nmtd_dropdown_day_plot,
            year_dropdown_day_plot,
            month_dropdown_day_plot,
            day_dropdown_day_plot,
        ]
        stat_widgets = [
            ctdl_dropdown_stat,
            nmtd_dropdown_stat,
            madiemdo_dropdown_stat,
            year_dropdown_stat,
            month_dropdown_stat,
        ]

        # Lưu vào dictionary để tái sử dụng
        ui_components = {
            "month_plot_widgets": month_plot_widgets,
            "day_plot_widgets": day_plot_widgets,
            "stat_widgets": stat_widgets,
            "placeholder_month_plot": placeholder_month_plot,
            "ctdl_dropdown_month_plot": ctdl_dropdown_month_plot,
            "nmtd_dropdown_month_plot": nmtd_dropdown_month_plot,
            "year_dropdown_month_plot": year_dropdown_month_plot,
            "month_dropdown_month_plot": month_dropdown_month_plot,
            "ctdl_dropdown_day_plot": ctdl_dropdown_day_plot,
            "nmtd_dropdown_day_plot": nmtd_dropdown_day_plot,
            "year_dropdown_day_plot": year_dropdown_day_plot,
            "month_dropdown_day_plot": month_dropdown_day_plot,
            "day_dropdown_day_plot": day_dropdown_day_plot,
            "ctdl_dropdown_stat": ctdl_dropdown_stat,
            "nmtd_dropdown_stat": nmtd_dropdown_stat,
            "madiemdo_dropdown_stat": madiemdo_dropdown_stat,
            "year_dropdown_stat": year_dropdown_stat,
            "month_dropdown_stat": month_dropdown_stat,
        }

        # ========== GẮN OBSERVE BAN ĐẦU ==========
        ctdl_dropdown_month_plot.observe(
            lambda change: update_nmtd(change["new"], nmtd_dropdown_month_plot),
            names="value",
        )
        ctdl_dropdown_day_plot.observe(
            lambda change: update_nmtd(change["new"], nmtd_dropdown_day_plot),
            names="value",
        )
        ctdl_dropdown_stat.observe(
            lambda change: update_nmtd(change["new"], nmtd_dropdown_stat), names="value"
        )
        nmtd_dropdown_stat.observe(
            lambda change: update_madiemdo(
                ctdl_dropdown_stat.value, change["new"], madiemdo_dropdown_stat
            ),
            names="value",
        )

        # Khởi tạo
        if ctdl_dropdown_month_plot.value:
            update_nmtd(ctdl_dropdown_month_plot.value, nmtd_dropdown_month_plot)
        if ctdl_dropdown_day_plot.value:
            update_nmtd(ctdl_dropdown_day_plot.value, nmtd_dropdown_day_plot)
        if ctdl_dropdown_stat.value:
            update_nmtd(ctdl_dropdown_stat.value, nmtd_dropdown_stat)
        if ctdl_dropdown_stat.value and nmtd_dropdown_stat.value:
            update_madiemdo(
                ctdl_dropdown_stat.value,
                nmtd_dropdown_stat.value,
                madiemdo_dropdown_stat,
            )

        year_dropdown_day_plot.observe(update_day_options, names="value")
        month_dropdown_day_plot.observe(update_day_options, names="value")
        if year_dropdown_day_plot.value and month_dropdown_day_plot.value:
            update_day_options()

        # ========== NHÓM OBSERVER ==========
        for w in month_plot_widgets:
            _observed_callbacks[w] = {"func": on_change_month_plot, "active": False}
        for w in day_plot_widgets:
            _observed_callbacks[w] = {"func": on_change_day_plot, "active": False}
        for w in stat_widgets:
            _observed_callbacks[w] = {"func": on_change_stat, "active": False}

        # ========== TẠO GIAO DIỆN TABS ==========
        tab_month_plot = widgets.HBox(
            month_plot_widgets + [placeholder_month_plot],
            layout=widgets.Layout(
                justify_content="flex-start", align_items="flex-end", overflow="visible"
            ),
        )
        tab_day_plot = widgets.HBox(
            day_plot_widgets,
            layout=widgets.Layout(
                justify_content="flex-start", align_items="flex-end", overflow="visible"
            ),
        )
        tab_stat = widgets.HBox(
            stat_widgets,
            layout=widgets.Layout(
                justify_content="flex-start", align_items="flex-end", overflow="visible"
            ),
        )
        tabs = widgets.Tab(children=[tab_month_plot, tab_day_plot, tab_stat])
        tabs.set_title(0, "📅 Biểu đồ tháng")
        tabs.set_title(1, "🗓️ Biểu đồ ngày")
        tabs.set_title(2, "📊 Thống kê tháng")

        out = widgets.Output()
        ui_components["tabs"] = tabs
        ui_components["out"] = out
        ui_components["tab_month_plot"] = tab_month_plot
        ui_components["tab_day_plot"] = tab_day_plot
        ui_components["tab_stat"] = tab_stat

        # Gắn xử lý chuyển tab
        tabs.observe(on_tab_change, names="selected_index")

        return ui_components
    else:
        print("\n❌ Dữ liệu không khả dụng hoặc rỗng.")
        return None


def show_ui():
    """Hiển thị giao diện tương tác"""
    global ui_components

    if "tabs" in ui_components and "out" in ui_components:
        print("✅ Giao diện sẵn sàng!")
        display(ui_components["tabs"], ui_components["out"])

        # Kích hoạt lần đầu
        on_tab_change(
            {
                "name": "selected_index",
                "old": None,
                "new": ui_components["tabs"].selected_index,
                "owner": ui_components["tabs"],
                "type": "change",
            }
        )
    else:
        print("❌ Giao diện chưa được khởi tạo.")


# ===== HÀM HỖ TRỢ =====
def make_dropdown(options, description, width="250px", margin="0px 20px 0px 0px"):
    """Tạo dropdown widget"""
    safe_options = list(options) if options is not None else []
    return widgets.Dropdown(
        options=safe_options,
        description=description,
        value=safe_options[0] if safe_options else None,
        layout=widgets.Layout(width=width, margin=margin),
        style={"description_width": "auto"},
        disabled=not bool(safe_options),
    )


def update_nmtd(ctdl_value, dropdown_to_update):
    """Cập nhật danh sách nhà máy theo CTDL"""
    global nmtd_cache, df_sanluong_indexed

    options = []
    current_nmtd_value = dropdown_to_update.value

    if ctdl_value:
        if ctdl_value in nmtd_cache:  # Kiểm tra cache trước
            options = nmtd_cache[ctdl_value]
        elif not df_sanluong_indexed.empty:
            try:
                # Lấy NMTD categories từ index con tương ứng với CTDL
                nmtd_options = (
                    df_sanluong_indexed.loc[ctdl_value]
                    .index.get_level_values("NMTD")
                    .unique()
                    .tolist()
                )
                options = sorted(nmtd_options)
                nmtd_cache[ctdl_value] = options  # Lưu vào cache
            except KeyError:
                options = []  # CTDL không có trong index
            except Exception as e:
                print(f"Lỗi cập nhật NMTD cache cho '{ctdl_value}': {e}")
                options = []

    # Cập nhật dropdown
    dropdown_to_update.options = options
    dropdown_to_update.disabled = not bool(options)
    if current_nmtd_value in options:
        dropdown_to_update.value = current_nmtd_value
    elif options:
        dropdown_to_update.value = options[0]
    else:
        dropdown_to_update.value = None


def update_madiemdo(ctdl_value, nmtd_value, dropdown_to_update):
    """Cập nhật danh sách mã điểm đo theo CTDL và NMTD"""
    global madiemdo_cache, df_sanluong

    options = []
    current_madiemdo_value = dropdown_to_update.value

    if ctdl_value and nmtd_value:
        cache_key = f"{ctdl_value}_{nmtd_value}"
        if cache_key in madiemdo_cache:
            options = madiemdo_cache[cache_key]
        elif not df_sanluong.empty:
            try:
                # Lấy danh sách các mã điểm đo của nhà máy đã chọn
                madiemdo_options = (
                    df_sanluong[
                        (df_sanluong["CTDL"] == ctdl_value)
                        & (df_sanluong["NMTD"] == nmtd_value)
                    ]["MADIEMDO"]
                    .unique()
                    .tolist()
                )
                options = sorted(madiemdo_options)
                madiemdo_cache[cache_key] = options  # Lưu vào cache
            except Exception as e:
                print(f"Lỗi cập nhật MADIEMDO cache cho '{cache_key}': {e}")
                options = []

    # Cập nhật dropdown
    dropdown_to_update.options = options
    dropdown_to_update.disabled = not bool(options)
    if current_madiemdo_value in options:
        dropdown_to_update.value = current_madiemdo_value
    elif options:
        dropdown_to_update.value = options[0]
    else:
        dropdown_to_update.value = None


def update_day_options(*args):
    """Cập nhật danh sách ngày theo năm và tháng"""
    global ui_components

    year = ui_components["year_dropdown_day_plot"].value
    month = ui_components["month_dropdown_day_plot"].value
    current_day = ui_components["day_dropdown_day_plot"].value
    day_options = []

    if year and month:
        try:
            max_day = calendar.monthrange(year, month)[1]
            day_options = list(range(1, max_day + 1))
        except Exception as e:
            print(f"Lỗi lấy số ngày {month}/{year}: {e}")

    ui_components["day_dropdown_day_plot"].options = day_options
    ui_components["day_dropdown_day_plot"].disabled = not bool(day_options)

    if current_day in day_options:
        ui_components["day_dropdown_day_plot"].value = current_day
    elif day_options:
        ui_components["day_dropdown_day_plot"].value = 1
    else:
        ui_components["day_dropdown_day_plot"].value = None


def on_change_month_plot(change):
    """Xử lý sự kiện thay đổi dropdown trong tab biểu đồ tháng"""
    global ui_components

    if ui_components["tabs"].selected_index == 0 and all(
        w.value is not None for w in ui_components["month_plot_widgets"]
    ):
        if change is None or (
            isinstance(change, dict) and change.get("new") != change.get("old")
        ):
            plot_filtered(
                "month",
                ui_components["ctdl_dropdown_month_plot"].value,
                ui_components["nmtd_dropdown_month_plot"].value,
                ui_components["year_dropdown_month_plot"].value,
                ui_components["month_dropdown_month_plot"].value,
            )


def on_change_day_plot(change):
    """Xử lý sự kiện thay đổi dropdown trong tab biểu đồ ngày"""
    global ui_components

    if ui_components["tabs"].selected_index == 1 and all(
        w.value is not None for w in ui_components["day_plot_widgets"]
    ):
        if change is None or (
            isinstance(change, dict) and change.get("new") != change.get("old")
        ):
            plot_filtered(
                "day",
                ui_components["ctdl_dropdown_day_plot"].value,
                ui_components["nmtd_dropdown_day_plot"].value,
                ui_components["year_dropdown_day_plot"].value,
                ui_components["month_dropdown_day_plot"].value,
                ui_components["day_dropdown_day_plot"].value,
            )


def on_change_stat(change):
    """Xử lý sự kiện thay đổi dropdown trong tab thống kê"""
    global ui_components

    if ui_components["tabs"].selected_index == 2 and all(
        w.value is not None for w in ui_components["stat_widgets"]
    ):
        if change is None or (
            isinstance(change, dict) and change.get("new") != change.get("old")
        ):
            plot_timeslot_stats(
                ui_components["ctdl_dropdown_stat"].value,
                ui_components["nmtd_dropdown_stat"].value,
                ui_components["madiemdo_dropdown_stat"].value,
                ui_components["year_dropdown_stat"].value,
                ui_components["month_dropdown_stat"].value,
            )


def disable_observe(widgets_list):
    """Tắt observer cho các widget"""
    global _observed_callbacks

    for w in widgets_list:
        if w in _observed_callbacks and _observed_callbacks[w]["active"]:
            try:
                w.unobserve(_observed_callbacks[w]["func"], names="value")
                _observed_callbacks[w]["active"] = False
            except:
                _observed_callbacks[w]["active"] = False


def enable_observe(widgets_list):
    """Bật observer cho các widget"""
    global _observed_callbacks

    for w in widgets_list:
        if (
            w in _observed_callbacks
            and not w.disabled
            and not _observed_callbacks[w]["active"]
        ):
            try:
                w.observe(_observed_callbacks[w]["func"], names="value")
                _observed_callbacks[w]["active"] = True
            except Exception as e:
                print(f"Lỗi observe {w.description}: {e}")


def on_tab_change(change):
    """Xử lý sự kiện chuyển tab"""
    global ui_components

    if change["name"] == "selected_index":
        new_tab_index = change["new"]
        old_tab_index = change["old"]

        # --- Tắt observer tab cũ ---
        if old_tab_index == 0:
            disable_observe(ui_components["month_plot_widgets"])
        elif old_tab_index == 1:
            disable_observe(ui_components["day_plot_widgets"])
        elif old_tab_index == 2:
            disable_observe(ui_components["stat_widgets"])

        # --- Xác định nguồn đồng bộ ---
        source_widgets = {}
        if old_tab_index == 0:
            source_widgets = {
                "ctdl": ui_components["ctdl_dropdown_month_plot"],
                "nmtd": ui_components["nmtd_dropdown_month_plot"],
                "year": ui_components["year_dropdown_month_plot"],
                "month": ui_components["month_dropdown_month_plot"],
            }
        elif old_tab_index == 1:
            source_widgets = {
                "ctdl": ui_components["ctdl_dropdown_day_plot"],
                "nmtd": ui_components["nmtd_dropdown_day_plot"],
                "year": ui_components["year_dropdown_day_plot"],
                "month": ui_components["month_dropdown_day_plot"],
                "day": ui_components["day_dropdown_day_plot"],
            }
        elif old_tab_index == 2:
            source_widgets = {
                "ctdl": ui_components["ctdl_dropdown_stat"],
                "nmtd": ui_components["nmtd_dropdown_stat"],
                "year": ui_components["year_dropdown_stat"],
                "month": ui_components["month_dropdown_stat"],
            }
        else:  # Khởi tạo
            source_widgets = {
                "ctdl": ui_components["ctdl_dropdown_month_plot"],
                "nmtd": ui_components["nmtd_dropdown_month_plot"],
                "year": ui_components["year_dropdown_month_plot"],
                "month": ui_components["month_dropdown_month_plot"],
            }

        # --- Xác định widgets đích của tab mới ---
        target_widgets = {}
        if new_tab_index == 0:
            target_widgets = {
                "ctdl": ui_components["ctdl_dropdown_month_plot"],
                "nmtd": ui_components["nmtd_dropdown_month_plot"],
                "year": ui_components["year_dropdown_month_plot"],
                "month": ui_components["month_dropdown_month_plot"],
            }
        elif new_tab_index == 1:
            target_widgets = {
                "ctdl": ui_components["ctdl_dropdown_day_plot"],
                "nmtd": ui_components["nmtd_dropdown_day_plot"],
                "year": ui_components["year_dropdown_day_plot"],
                "month": ui_components["month_dropdown_day_plot"],
                "day": ui_components["day_dropdown_day_plot"],
            }
        elif new_tab_index == 2:
            target_widgets = {
                "ctdl": ui_components["ctdl_dropdown_stat"],
                "nmtd": ui_components["nmtd_dropdown_stat"],
                "madiemdo": ui_components["madiemdo_dropdown_stat"],
                "year": ui_components["year_dropdown_stat"],
                "month": ui_components["month_dropdown_stat"],
            }

        # --- Đồng bộ từ nguồn sang đích ---
        if old_tab_index != new_tab_index and old_tab_index is not None:
            # Đồng bộ CTDL
            if source_widgets.get("ctdl") and source_widgets["ctdl"].value is not None:
                target_widgets["ctdl"].value = source_widgets["ctdl"].value

            # Đồng bộ NMTD (phải cập nhật options trước)
            target_nmtd = target_widgets["nmtd"]
            update_nmtd(target_widgets["ctdl"].value, target_nmtd)
            if (
                source_widgets.get("nmtd")
                and source_widgets["nmtd"].value in target_nmtd.options
            ):
                target_nmtd.value = source_widgets["nmtd"].value

            # Đồng bộ MADIEMDO (phải cập nhật options trước)
            if new_tab_index == 2 and "madiemdo" in target_widgets:
                update_madiemdo(
                    target_widgets["ctdl"].value,
                    target_widgets["nmtd"].value,
                    target_widgets["madiemdo"],
                )

            # Đồng bộ Year
            if source_widgets.get("year") and source_widgets["year"].value is not None:
                target_widgets["year"].value = source_widgets["year"].value

            # Đồng bộ Month
            if (
                source_widgets.get("month")
                and source_widgets["month"].value is not None
            ):
                target_widgets["month"].value = source_widgets["month"].value

            # Nếu chuyển sang tab ngày, cập nhật dropdown ngày
            if "day" in target_widgets:
                update_day_options()  # Cập nhật options ngày dựa trên năm/tháng đã chọn
                # Mặc định chọn ngày 1 nếu đến từ tab khác
                if old_tab_index != 1 and target_widgets["day"].options:
                    target_widgets["day"].value = 1

        # --- Kích hoạt observer và cập nhật nội dung tab mới ---
        if new_tab_index == 0:
            enable_observe(ui_components["month_plot_widgets"])
            on_change_month_plot(None)
        elif new_tab_index == 1:
            enable_observe(ui_components["day_plot_widgets"])
            on_change_day_plot(None)
        elif new_tab_index == 2:
            enable_observe(ui_components["stat_widgets"])
            on_change_stat(None)


def plot_filtered(mode, ctdl, nmtd, year, month, day=None):
    """Vẽ biểu đồ theo các tham số lọc"""
    global df_sanluong, ui_components

    with ui_components["out"]:
        clear_output(wait=True)
        plot_start_time = time.time()

        # Kiểm tra đầu vào
        required_vals = [ctdl, nmtd, year, month]
        if mode == "day":
            required_vals.append(day)
        if not all(v is not None for v in required_vals):
            print("   ⚠️ Vui lòng chọn đủ các bộ lọc.")
            return

        filtered_df = pd.DataFrame()
        try:
            # Tạo mask boolean hiệu quả
            mask = (
                (df_sanluong["CTDL"] == ctdl)
                & (df_sanluong["NMTD"] == nmtd)
                & (df_sanluong["YEAR"] == year)
                & (df_sanluong["MONTH_NUM"] == month)
            )
            if mode == "day":
                mask &= df_sanluong["DAY_NUM"] == day

            # Chỉ lấy các cột cần thiết cho vẽ
            filtered_df = df_sanluong.loc[mask, ["TIME", "CS", "MADIEMDO"]].copy()
            # Sắp xếp theo TIME
            filtered_df.sort_values("TIME", inplace=True)

        except Exception as e:
            print(f"❌ Lỗi khi lọc dữ liệu cho biểu đồ: {e}")

        if filtered_df.empty:
            print("   ⚠️ Không có dữ liệu cho lựa chọn này!")
            return

        # Vẽ biểu đồ
        try:
            title_ext = (
                f"{month}/{year}" if mode == "month" else f"{day}/{month}/{year}"
            )
            fig = px.line(
                filtered_df,
                x="TIME",
                y="CS",
                color="MADIEMDO",
                render_mode="webgl",
                markers=True if mode == "day" else False,
            )

            # Áp dụng layout
            fig.update_layout(
                title=None,
                annotations=[
                    dict(
                        text=f"<b>Công suất theo chu kỳ 30' - {nmtd} - {title_ext}</b>",
                        xref="paper",
                        yref="paper",
                        x=0.45,
                        y=1.1,
                        xanchor="center",
                        yanchor="top",
                        showarrow=False,
                        font=dict(size=18, color="black"),
                    )
                ],
                template="plotly_white",
                height=600,
                hovermode="x unified",
                xaxis_title=None,
                yaxis_title="Giá trị công suất",
                xaxis=dict(
                    showgrid=True,
                    gridcolor="lightgrey",
                    griddash="dot",
                    title_font=dict(size=14, color="black"),
                    ticklabelposition="outside",
                    ticks="outside",
                    ticklen=8,
                ),
                yaxis=dict(
                    showgrid=True,
                    gridcolor="lightgrey",
                    griddash="dot",
                    rangemode="tozero",
                    title_font=dict(size=14, color="black", weight="bold"),
                    ticklabelposition="outside",
                    ticks="outside",
                    ticklen=8,
                ),
                margin=dict(l=60, r=40, t=80, b=80),
                paper_bgcolor="white",
                plot_bgcolor="#f0f8ff",
            )
            fig.add_annotation(
                text="<b>Thời gian</b>",
                xref="paper",
                yref="paper",
                x=0.45,
                y=-0.15,
                showarrow=False,
                font=dict(size=14, color="black"),
            )
            fig.update_traces(
                line=dict(width=2),
                hovertemplate="Công suất: %{y}",
            )

            fig.show()

        except Exception as e:
            print(f"❌ Lỗi khi vẽ biểu đồ Plotly Express: {e}")


def plot_timeslot_stats(ctdl, nmtd, madiemdo, year, month):
    """Vẽ biểu đồ thống kê theo khung giờ"""
    global df_sanluong_indexed, ui_components

    with ui_components["out"]:
        clear_output(wait=True)

        if not all([ctdl, nmtd, year, month]):
            print("   ⚠️ Vui lòng chọn đủ CTDL, Nhà máy, Năm và Tháng.")
            return

        stats_by_time = pd.DataFrame()
        try:
            # Dùng .loc trên df_sanluong_indexed
            idx_query = (ctdl, nmtd, madiemdo, year, month)
            # Kiểm tra sự tồn tại của index
            if idx_query in df_sanluong_indexed.index.droplevel("DAY_NUM").unique():
                monthly_data = df_sanluong_indexed.loc[idx_query, ["TIME_SLOT", "CS"]]

                if not monthly_data.empty:
                    # Group và Agg
                    stats_by_time = monthly_data.groupby("TIME_SLOT", observed=True)[
                        "CS"
                    ].agg(
                        min="min",
                        p25=lambda x: x.quantile(0.25),
                        p50="mean",
                        p75=lambda x: x.quantile(0.75),
                        max="max",
                    )
                    # Sắp xếp index
                    stats_by_time = stats_by_time.sort_index()
                else:
                    print("   ℹ️ Không có dữ liệu chi tiết sau khi lọc.")

        except KeyError:
            print("   ℹ️ Không tìm thấy dữ liệu (KeyError).")
        except Exception as e:
            print(f"❌ Lỗi tính toán thống kê: {e}")

        if stats_by_time.empty:
            print("   ℹ️ Không có dữ liệu cho lựa chọn này!")
            return

        # Vẽ biểu đồ
        try:
            fig = go.Figure()
            time_slots = stats_by_time.index
            p25 = stats_by_time["p25"]
            p75 = stats_by_time["p75"]
            p50 = stats_by_time["p50"]
            min_cs = stats_by_time["min"]
            max_cs = stats_by_time["max"]

            # 1. Vùng Min-Max
            fig.add_trace(
                go.Scatter(
                    x=time_slots,
                    y=max_cs,
                    mode="lines",
                    line=dict(width=0),
                    showlegend=False,
                    hoverinfo="skip",
                    hovertemplate="Max: %{y:.2f}<extra></extra>",
                )
            )
            fig.add_trace(
                go.Scatter(
                    x=time_slots,
                    y=min_cs,
                    mode="lines",
                    line=dict(width=0),
                    fill="tonexty",
                    fillcolor="rgba(211, 211, 211, 0.5)",
                    name="Min-Max",
                    hoverinfo="skip",
                    hovertemplate="Min: %{y:.2f}<extra></extra>",
                )
            )

            # 2. Vùng P25-P75
            fig.add_trace(
                go.Scatter(
                    x=time_slots,
                    y=p75,
                    mode="lines",
                    line=dict(width=0),
                    showlegend=False,
                    hoverinfo="skip",
                    hovertemplate="P75: %{y:.2f}<extra></extra>",
                )
            )
            fig.add_trace(
                go.Scatter(
                    x=time_slots,
                    y=p25,
                    mode="lines",
                    line=dict(width=0),
                    fill="tonexty",
                    fillcolor="rgba(173, 216, 230, 0.6)",
                    name="Tần suất 25-75",
                    hoverinfo="skip",
                    hovertemplate="P25: %{y:.2f}<extra></extra>",
                )
            )

            # 3. Đường Median (P50)
            fig.add_trace(
                go.Scatter(
                    x=time_slots,
                    y=p50,
                    mode="lines+markers",
                    marker=dict(size=6, color="blue"),
                    line=dict(color="blue", width=2.5),
                    name="Trung bình",
                    hovertemplate="Mean: %{y:.2f}<extra></extra>",
                )
            )

            # Layout
            fig.update_layout(
                title=None,
                annotations=[
                    dict(
                        text=f"<b>Thống kê công suất theo khung giờ - {nmtd} - {month}/{year}</b>",
                        xref="paper",
                        yref="paper",
                        x=0.45,
                        y=1.1,
                        xanchor="center",
                        yanchor="top",
                        showarrow=False,
                        font=dict(size=18, color="black"),
                    )
                ],
                template="plotly_white",
                height=600,
                hovermode="x unified",
                xaxis_title=None,
                yaxis_title="Giá trị công suất",
                xaxis=dict(
                    tickmode="array",
                    tickvals=time_slots,
                    ticktext=[t for t in time_slots],
                    showgrid=True,
                    gridcolor="lightgrey",
                    griddash="dot",
                    title_font=dict(size=14, color="black"),
                    ticklabelposition="outside",
                    ticks="outside",
                    ticklen=8,
                    tickangle=-90,
                ),
                yaxis=dict(
                    rangemode="tozero",
                    showgrid=True,
                    gridcolor="lightgrey",
                    griddash="dot",
                    title_font=dict(size=14, color="black", weight="bold"),
                    ticklabelposition="outside",
                    ticks="outside",
                    ticklen=8,
                ),
                legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99),
                margin=dict(l=60, r=40, t=80, b=80),
                paper_bgcolor="white",
                plot_bgcolor="#f0f8ff",
            )
            fig.add_annotation(
                text="<b>Khung giờ</b>",
                xref="paper",
                yref="paper",
                x=0.45,
                y=-0.18,
                showarrow=False,
                font=dict(size=14, color="black"),
            )
            fig.show()

        except Exception as e:
            print(f"❌ Lỗi vẽ biểu đồ thống kê Plotly GO: {e}")


# Hàm chạy toàn bộ quy trình một lần
def run_full_process(parquet_folder=parquet_folder):
    """Chạy toàn bộ quy trình đọc dữ liệu, xử lý và hiển thị giao diện"""
    read_data(parquet_folder)
    preprocess_data()
    create_ui()
    show_ui()


# Hàm tùy chỉnh chỉ hiển thị UI (nếu dữ liệu đã được xử lý)
def reload_ui():
    """Tải lại UI từ dữ liệu đã xử lý"""
    global df_sanluong, df_sanluong_indexed

    if not df_sanluong.empty and not df_sanluong_indexed.empty:
        create_ui()
        show_ui()
    else:
        print("❌ Cần phải đọc và xử lý dữ liệu trước.")


# Khởi chạy quy trình khi cell được chạy
if __name__ == "__main__":
    # Nếu chạy lần đầu tiên
    if df_all.empty:
        run_full_process()
    # Nếu đã có dữ liệu và chỉ cần tải lại UI
    elif not df_sanluong.empty and not df_sanluong_indexed.empty:
        reload_ui()
    else:
        print("⚠️ Dữ liệu đã được đọc nhưng chưa xử lý.")
        preprocess_data()
        create_ui()
        show_ui()

🔄 Bắt đầu đọc dữ liệu Parquet...
🔍 Tìm thấy 4 file Parquet. Bắt đầu đọc...
   Ghép các DataFrame...
✅ Đọc và ghép 4 file thành công.
👉 Tổng số dòng: 18,404,864
⏱️ Thời gian đọc và ghép: 18.78 giây

🔄 Bắt đầu tiền xử lý và tối ưu hóa...
   ⚠️ Đã loại bỏ 60,706 dòng có thời gian không theo chuẩn 30 phút.
   ✅ DataFrame gốc đã được xử lý và tối ưu hóa kiểu dữ liệu.
   ✅ DataFrame index cho thống kê đã được tạo và sắp xếp.
⏱️ Thời gian tiền xử lý: 150.87 giây

🔄 Tạo giao diện tương tác...
✅ Giao diện sẵn sàng!


Output()

In [21]:
df_sanluong[df_sanluong["MADIEMDO"] == "G2A011S000M132"].sort_values(
    by="TIME", ascending=True
)

,CTDL,NMTD,MADIEMDO,TIME,CS,YEAR,MONTH_NUM,DAY_NUM,TIME_SLOT
197111,CTY ĐIỆN LỰC SƠN LA,NMTĐ NẬM LA,G2A011S000M132,2021-01-01 00:30:00,0.0,2021,1,1,00:30
197112,CTY ĐIỆN LỰC SƠN LA,NMTĐ NẬM LA,G2A011S000M132,2021-01-01 01:00:00,0.0,2021,1,1,01:00
197113,CTY ĐIỆN LỰC SƠN LA,NMTĐ NẬM LA,G2A011S000M132,2021-01-01 01:30:00,0.0,2021,1,1,01:30
197114,CTY ĐIỆN LỰC SƠN LA,NMTĐ NẬM LA,G2A011S000M132,2021-01-01 02:00:00,0.0,2021,1,1,02:00
197115,CTY ĐIỆN LỰC SƠN LA,NMTĐ NẬM LA,G2A011S000M132,2021-01-01 02:30:00,0.0,2021,1,1,02:30
...,...,...,...,...,...,...,...,...,...
17703435,CTY ĐIỆN LỰC SƠN LA,NMTĐ NẬM LA,G2A011S000M132,2024-08-07 03:00:00,120.0,2024,8,7,03:00
17703436,CTY ĐIỆN LỰC SƠN LA,NMTĐ NẬM LA,G2A011S000M132,2024-08-07 03:30:00,130.0,2024,8,7,03:30
17703437,CTY ĐIỆN LỰC SƠN LA,NMTĐ NẬM LA,G2A011S000M132,2024-08-07 04:00:00,120.0,2024,8,7,04:00
17703438,CTY ĐIỆN LỰC SƠN LA,NMTĐ NẬM LA,G2A011S000M132,2024-08-07 04:30:00,130.0,2024,8,7,04:30


In [36]:
df_check = df_sanluong[
    (df_sanluong["NMTD"] == "NẬM SO 1")].sort_values(by="TIME", ascending=True)

In [37]:
df_check

,CTDL,NMTD,MADIEMDO,TIME,CS,YEAR,MONTH_NUM,DAY_NUM,TIME_SLOT
4279188,CTY ĐIỆN LỰC LAI CHÂU,NẬM SO 1,G2A238S000M901,2021-12-01 00:30:00,0.0,2021,12,1,00:30
4279189,CTY ĐIỆN LỰC LAI CHÂU,NẬM SO 1,G2A238S000M901,2021-12-01 01:00:00,0.0,2021,12,1,01:00
4279190,CTY ĐIỆN LỰC LAI CHÂU,NẬM SO 1,G2A238S000M901,2021-12-01 01:30:00,0.0,2021,12,1,01:30
4279191,CTY ĐIỆN LỰC LAI CHÂU,NẬM SO 1,G2A238S000M901,2021-12-01 02:00:00,0.0,2021,12,1,02:00
4279192,CTY ĐIỆN LỰC LAI CHÂU,NẬM SO 1,G2A238S000M901,2021-12-01 02:30:00,0.0,2021,12,1,02:30
...,...,...,...,...,...,...,...,...,...
16167674,CTY ĐIỆN LỰC LAI CHÂU,NẬM SO 1,G2A238S000M901,2024-08-08 12:00:00,70.0,2024,8,8,12:00
16167693,CTY ĐIỆN LỰC LAI CHÂU,NẬM SO 1,G2A238S000M902,2024-08-08 12:30:00,60.0,2024,8,8,12:30
16167692,CTY ĐIỆN LỰC LAI CHÂU,NẬM SO 1,G2A238S000M901,2024-08-08 12:30:00,60.0,2024,8,8,12:30
16167676,CTY ĐIỆN LỰC LAI CHÂU,NẬM SO 1,G2A238S000M901,2024-08-08 13:00:00,60.0,2024,8,8,13:00


Trong đoạn code sau trong phần xử lý dữ liệu, do trong cột 'ENDTIME' của df_all ban đầu có nhiều điểm dữ liệu bị lỗi không thuộc những giá trị thời gian có dạng cập nhật 30 phút một lần như 2024-07-09 09:30:00, 2024-07-09 10:00:00... mà lại có dạng 2024-07-05 09:10:11 hay 2024-07-05 17:40:14. Tôi muốn bạn hãy bổ sung phần xóa bỏ những khoảng thời gian bị lỗi này khi xử lý dữ liệu ban đầu